In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import BaggingClassifier

In [23]:
df = pd.read_csv('wineQualityReds.zip', index_col='Unnamed: 0')
df.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [24]:
df['label'] = df['quality'].apply(lambda x: 1 if x > 5 else 0 )
df['label']

1       0
2       0
3       0
4       1
5       0
       ..
1595    0
1596    1
1597    1
1598    0
1599    1
Name: label, Length: 1599, dtype: int64

In [25]:
# Разделим выборку на обучающую и тестовую в соотношении 70/30 с random_state=42
X = df.drop(['quality', 'label'], axis=1)  # все столбцы кроме 'quality' и 'label' используем как признаки
y = df['label']  # наша метка
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [26]:
# Теперь обучим два классификатора: логистическую регрессию и дерево решений
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train, y_train)


c:\Python 3.9.10\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [27]:

decision_tree_model = DecisionTreeClassifier(random_state=42, max_depth=10)
decision_tree_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=10, random_state=42)

In [29]:
# Применяем модели к тестовым данным
logistic_regression_predictions = logistic_regression_model.predict(X_test)
decision_tree_predictions = decision_tree_model.predict(X_test)

# Вычисляем F1-меру для каждой модели
f1_logistic_regression = f1_score(y_test, logistic_regression_predictions)
f1_decision_tree = f1_score(y_test, decision_tree_predictions)

# Выбираем модель с лучшим значением F1-меры
best_f1_score = max(f1_logistic_regression, f1_decision_tree)

# Выводим значение F1-меры для лучшей модели, округленное до трех знаков после точки-разделителя
print(round(best_f1_score, 3))
print(f1_logistic_regression,)

0.793
0.75


In [31]:
# Используем дерево решений в качестве базового классификатора
bagging_model = BaggingClassifier(base_estimator=decision_tree_model, n_estimators=1500, random_state=42)

# Обучаем модель
bagging_model.fit(X_train, y_train)

# Вычисляем F1-меру для модели с использованием бэггинга
bagging_predictions = bagging_model.predict(X_test)
f1_bagging = f1_score(y_test, bagging_predictions)

# Выводим новое значение F1-меры
print(round(f1_bagging, 3))

c:\Python 3.9.10\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.818
